<a href="https://colab.research.google.com/github/kamranuz/mmdm2024-lab-1/blob/master/HSE_MMDM_2024_%7C_%D0%9B%D0%B0%D0%B1%D0%B0%D1%80%D0%BE%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%E2%84%961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import random
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm import tqdm
from deap import base, creator, tools, algorithms

# HSE MMDM 2024 | Lab Work №1


-- Rules --
- Grade = 0.3*Task1 + 0.3*Task2 + 0.4*Task3
- You can work in small groups: 1-3 students.
- Correct but basic solution of each task costs 8 points. To get 9, 10 be creative: think of some modification for basic algorithms; cool visualisation may also give you additional points
- Write your own code for algorithms!
- Deadline - TBD


-- Team --

[Mike](https://t.me/MC_Mikel) &
[Вацлав](https://t.me/RinokuS) &
[Kamrаn](https://t.me/kamran_uz)


### **Tickets**
| Title | Explanation | Responsible Person |[_]|
| --- | --- | --- |--- |
| Preparation | Prepare space/material and etc for team work | Kamron | [x]|
| Preparation | Create base classes for solutions | Вацлав |[_]|
| Task 1 | Implement baseline-solution using special libraries | Вацлав |[_]|
| Task 1 | Implement GA algorithm solution | Kamron |[_]|
| Task 1 | Make comprehensive analysis of the implementations | Mike |[_]|
| Task 1 | Create comprehensive visualizations | Вацлав |[_]|
| Task 2 | Implement baseline-solution using special libraries | Kamron |[_]|
| Task 2 | Implement NSGA II algorithm solution | Mike |[_]|
| Task 2 | Make comprehensive analysis of the implementations | Вацлав |[_]|
| Task 2 | Create comprehensive visualizations | Mike |[_]|
| Task 3 | TBD | ? |[_]|
| Post solution | Conclusions | Mike, Kamron, Вацлав|[_]|
| Post solution | Submission | Kamron |[_]|

## Task 1
Use genetic algorithm to find global extremum of given functions. Make sure your algorithm works for n dimensional objective functions.

(a)
$$
\max_{x,y} f(x, y)
=
3(1-x)^2
\exp\left(-x^2-(y+1)^2\right)
-
10\left(\frac{x}{5} - x^3 - y^5\right)
\exp\left(-x^2-y^2\right)
-
\frac{1}{3} \exp\left(-(x+1)^2-y^2\right)
$$
(b)

$$
\min_{x,y} f(x, y)
=
-0.0001
\left(
    \left\lvert
        \sin(x)
        \sin(y)
        \exp\left(\left|
            100 - \frac{\sqrt{x^2 + y^2}}{\pi}
            \right|\right)
    \right\rvert
    +
    1
\right)^{0.1}
\quad
(x, y) \in [-10, 10]^2
$$

In [2]:
## Task 1

f_a = lambda x, y: 3*(1-x)**2 * math.exp(-x**2-(y+1)**2) - 10*(x/5 - x**3 - y**5) * math.exp(-x**2-y**2) - 1/3 * math.exp(-(x+1)**2-y**2)
f_b = lambda x, y: -0.0001 * (abs(math.sin(x) * math.sin(y) * math.exp(abs(100 - math.sqrt(x**2 + y**2)/math.pi)) + 1)**0.1)

In [32]:
# @title baseline solution with libraries by Vatslav

def evaluate_a(individual):
    x, y = individual
    return f_a(x, y),

def evaluate_b(individual):
    x, y = individual
    return f_b(x, y),

def perform_genetic_algorithm(evaluate_function, handler="maximization", generations=100, size=100, verbose=False):
    weights = (1.0,) if handler == "maximization" else (-1.0,)

    try:
        del creator.Fitness
        del creator.Individual
    except Exception as e:
        pass

    creator.create("Fitness", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.Fitness)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, -100, 100)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_function)

    population = toolbox.population(n=size)
    algorithms.eaMuPlusLambda(population, toolbox, mu=50, lambda_=200, cxpb=0.6, mutpb=0.3, ngen=generations, verbose=verbose)

    # Get the best individual and its fitness value
    best_ind = tools.selBest(population, 1)[0]
    best_value = f_a(*best_ind)

    return best_ind, best_value

In [41]:
individual_a, value_a = perform_genetic_algorithm(evaluate_a)
print(f"For function f_a DEAP algo found best individual '{individual_a}' with function value '{value_a}'")

individual_b, value_b = perform_genetic_algorithm(evaluate_b, handler="minimization")
print(f"For function f_b DEAP algo found best individual '{individual_b}' with function value '{value_b}'")

For function f_a DEAP algo found best individual '[-0.009317589860859198, 1.5813679672836887]' with function value '8.10621358944234'
For function f_b DEAP algo found best individual '[1.349300561905505, -1.3480641786620327]' with function value '-0.5436560424404089'


In [27]:
# @title GA algorithm baseline by Kamron
def generate_population(points, size=100):
    n  = points.shape[0]
    identity = np.eye(n)
    population = [identity[np.random.permutation(range(n))] for _ in range(size)]
    return population

def crossover(population, add_size=100):
    popultaion_pairs_sample_id =  np.random.choice(range(len(population)),(add_size,2))
    next_population = [population[id_f]@population[id_m] for id_f, id_m in popultaion_pairs_sample_id]
    next_population += population
    return next_population

def swap_random_genes(organism):
    organism = organism.copy()
    id = range(len(organism))
    id_x, id_y = np.random.choice(id, 2)
    organism[[id_x, id_y]] = organism[[id_y, id_x]]
    return organism

def mutate(population, add_size=100):
    next_population = []
    popultaion_sample_id =  np.random.choice(range(len(population)),add_size)
    next_population = [swap_random_genes(population[organism_id]) for organism_id in popultaion_sample_id]
    next_population += population
    return next_population


def f_fitness(organism, points):
    gens_temp_calc = (organism-np.roll(organism,1,axis=0))@points
    fit_num = sum(np.sqrt(np.sum(gens_temp_calc*gens_temp_calc,axis=1)))
    return fit_num

def selection(population, points, size=100):
    selection_propabilty_temp = np.array([f_fitness(organism, points) for organism in population])

    selection_propabilty_temp = max(selection_propabilty_temp)-selection_propabilty_temp
    selection_propabilty      = selection_propabilty_temp/sum(selection_propabilty_temp)

    population_id = range(len(population))
    selected_population_id = np.random.choice(population_id,size,replace=False,p=selection_propabilty)
    selected_population = np.array(population)[selected_population_id]
    selected_population = list(selected_population)
    return selected_population

def genetic_algorithm(points, generations=100, size=100, crossover_size=100, mutate_size=10):
    log = {
        'generation':[],
        'min': [],
        'mean': [],
        'max': [],
        'population': []
    }
    population = generate_population(points, size=size)
    for i in tqdm(range(generations)):
        selection_propabilty_temp = np.array([f_fitness(organism, points) for organism in population])
        log['generation'] += [i]
        log['min']        += [min(selection_propabilty_temp)]
        log['mean']       += [np.mean(selection_propabilty_temp)]
        log['max']        += [max(selection_propabilty_temp)]
        log['population'] += [population]

        population = mutate(population, add_size=mutate_size)
        population = crossover(population, add_size=crossover_size)
        population = selection(population, points, size=size)
    return log, population

log, answer = genetic_algorithm(points, generations=100, size=100, crossover_size=100, mutate_size=10)

AttributeError: 'function' object has no attribute 'shape'

## Task 2

Find pareto front for the given multi-objective optimisation problem using NSGA II algorithm. Take n = 3, but you may try other values as well.
$$
\min_{x} = \begin{cases}
    f_1(x)
    =
    1
    -
    \exp\left(
        -\sum_{i=1}^{n}
            \left(
                x_i - \sqrt{\frac{1}{n}}
            \right)^2
        \right) \\
    f_2(x)
    =
    1
    -
    \exp\left(
        -\sum_{i=1}^{n}
            \left(
                x_i + \sqrt{\frac{1}{n}}
            \right)^2
        \right)
\end{cases}
\quad
-4 \leq x_i \leq 4, \quad 1 \leq i \leq n
$$


In [ ]:
## Task 2

# head-on implementation
f_1 = lambda x, n: 1 - math.exp(-sum((xi - math.sqrt(1/n))**2 for xi in x))
f_2 = lambda x, n: 1 - math.exp(-sum((xi + math.sqrt(1/n))**2 for xi in x))


# numpy implementation
f_1 = lambda x, n: 1 - np.exp(-np.sum((x - np.sqrt(1/n))**2))
f_2 = lambda x, n: 1 - np.exp(-np.sum((x + np.sqrt(1/n))**2))

## Task 3

TBD

In [ ]:
## Task 3